In [1]:
#!pip install ocs-academic-hub
#!pip uninstall -y ocs_academic_hub
#!pip list

## Note this notebook requires `omf_client.py` found at https://github.com/cfoisy-osisoft/academic-hub/tree/master/eds/tests

In [11]:
from ocs_academic_hub import OMFClient
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from dateutil.parser import parse
import time 
from time import sleep
from omf_client import OMFClient

# Data frame generation

In [12]:
start = pd.to_datetime("2021-04-01 15:00:00")
end = pd.to_datetime("2021-04-01 15:02:00")  # data frame worth 2 minutes of data
freq = 10  #  Hz
column = 4
# batch_size = 50 ==> replaced by batch_period, see "Parameters"

In [13]:
time_range = pd.date_range(start, end, freq=f"{1/freq*1000000000}ns")

np.random.seed(seed=1111)
data = np.random.uniform(1, high=100, size=(len(time_range), column))

df = pd.DataFrame({"Timestamp": time_range})
# df = df.set_index('Timestamp')
i = 0
while i < column:
    df[f"col{i + 1}"] = data[:, i]
    i += 1
print(df)

                   Timestamp       col1       col2       col3       col4
0    2021-04-01 15:00:00.000  10.459371  92.575366  35.013769  31.737217
1    2021-04-01 15:00:00.100   1.198974  24.323878  24.541380  73.855671
2    2021-04-01 15:00:00.200  50.051340  78.658110  13.524125  61.058283
3    2021-04-01 15:00:00.300  47.145977  24.476080  44.080759  25.123480
4    2021-04-01 15:00:00.400  39.000151  84.000975  65.863288  15.696221
...                      ...        ...        ...        ...        ...
1196 2021-04-01 15:01:59.600  15.595843  35.654702  64.807483  85.553442
1197 2021-04-01 15:01:59.700  10.539654   9.850069  10.691247  47.797944
1198 2021-04-01 15:01:59.800  17.147957  32.205425  76.982448   6.853298
1199 2021-04-01 15:01:59.900  75.652143  55.745506   3.989452  52.394992
1200 2021-04-01 15:02:00.000  28.305467  21.287744  32.430184  77.999849

[1201 rows x 5 columns]


In [14]:
# first row 
df.loc[0]

Timestamp    2021-04-01 15:00:00
col1                     10.4594
col2                     92.5754
col3                     35.0138
col4                     31.7372
Name: 0, dtype: object

## return a new dataframe where all timestamp are shifted with first row set to "now"

In [15]:
def shift_df_to_now(df):
    now = datetime.now()
    time_delta = now - df.loc[0]["Timestamp"]
    print(">> time delta=", time_delta)
    df["Timestamp"] = df["Timestamp"].apply(lambda t: t + time_delta)
    return df

In [16]:
tdf = shift_df_to_now(df)
tdf.index[-1]

>> time delta= 19 days 22:01:29.815798


1200

## To keep track of last row index sent from dataframe 

In [17]:
class Cursor:
    def __init__(self, interval=2):
        self._last_slice_index = -1
        self._interval = interval

    def last_index(self):
        return self._last_slice_index

    def set_last_index(self, index):
        self._last_slice_index = index

    def period(self):
        return self._interval

    def reset(self):
        self._last_slice_index = -1

## Extract next slice of data frame to transmit

In [18]:
def next_slice(df, cursor):
    now = datetime.now()
    slice_df = df[(df.index > cursor.last_index()) & (df["Timestamp"] <= now)]
    # print(f"row={len(slice_df)}")
    # print(slice_df)
    if len(slice_df) == 0:
        raise StopIteration
    cursor.set_last_index(slice_df.index[-1] + 1)
    return slice_df

## Parameters

In [19]:
batch_period = 2 # in seconds
asset_name = "test-asset1"
debug = True  # when active, OMF messages are only displayed  

## Main loop

The next cell can be restarted anytime because `shift_df_to_now` will make the dataframe "current"

In [20]:
cursor = Cursor(batch_period)
tdf = shift_df_to_now(df)

print(f"len={len(tdf)}")

omf = OMFClient("-no-key-", debug=debug)

omf.create_streams(tdf, asset_name, debug=debug)

try: 
    while True:
        time.sleep(batch_period)
        slice = next_slice(tdf, cursor)
        # print(
        #     "---\n",
        #    slice.iloc[0]["Timestamp"],
        #    "\n",
        #    slice.iloc[len(slice) - 1]["Timestamp"],
        #    len(slice),
        # )
        omf.update_streams(slice, debug=debug)
except StopIteration:
    print("--- done ---")

>> time delta= 0 days 00:00:03.686619
len=1201

DBG>>> http://localhost:5590/ap... {'producertoken': 'CSM_Test.EDS1', 'messagetype': 'type', 'messageformat': 'json', 'omfversion': '1.0', 'Ocp-Apim-Subscription-Key': '-no-key-'} [{'id': 'LabVIEW.Double', 'description': 'Timestamp and real-time value', 'type': 'object', 'classification': 'dynamic', 'properties': {'Timestamp': {'type': 'string', 'format': 'date-time', 'isindex': True}, 'value': {'type': 'Double'}}}]
>> [OMF type definition OK]

>> new tag(s): ['CSM_Test.EDS1.test-asset1.col1', 'CSM_Test.EDS1.test-asset1.col2', 'CSM_Test.EDS1.test-asset1.col3', 'CSM_Test.EDS1.test-asset1.col4']
>> from 2021-04-21 13:01:33.502417 to 2021-04-21 13:03:33.502417

DBG>>> http://localhost:5590/ap... {'producertoken': 'CSM_Test.EDS1', 'messagetype': 'container', 'messageformat': 'json', 'omfversion': '1.0', 'Ocp-Apim-Subscription-Key': '-no-key-'} [{'id': 'test-asset1.col1', 'typeid': 'LabVIEW.Double'}, {'id': 'test-asset1.col2', 'typeid': 'LabVI